In [48]:
import pandas as pd

In [49]:
data = pd.read_csv("C:\\Tweets.csv")

In [50]:
print(data.columns)
print(data.describe())
print(data['airline_sentiment'].value_counts())

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')
           tweet_id  airline_sentiment_confidence  negativereason_confidence  \
count  1.464000e+04                  14640.000000               10522.000000   
mean   5.692184e+17                      0.900169                   0.638298   
std    7.791112e+14                      0.162830                   0.330440   
min    5.675883e+17                      0.335000                   0.000000   
25%    5.685592e+17                      0.692300                   0.360600   
50%    5.694779e+17                      1.000000                   0.670600   
75%    5.698905e+17                      1.000000                   1.000000   
max    5.703106e+17           

In [51]:
data['text'] = data['text'].str.replace(r'\W', ' ')

In [52]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
top_words = data.groupby('airline')['text'].apply(lambda x: pd.Series(' '.join(x).lower().split()).value_counts().head(10))
print(top_words)

airline                       
American        @americanair      2706
                to                1734
                the               1170
                i                 1113
                a                  836
                and                797
                for                773
                you                741
                on                 713
                my                 649
Delta           @jetblue          2007
                to                1106
                the                848
                i                  743
                a                  558
                on                 526
                for                464
                and                453
                you                449
                flight             420
Southwest       @southwestair     2362
                to                1379
                the                947
                i                  884
                for              

In [54]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['text'])
print(X_train_tfidf)
print(X_test_tfidf)

  (0, 5676)	0.18332050200557587
  (0, 3350)	0.3125789825606384
  (0, 7178)	0.33824032632007733
  (0, 430)	0.26283191552540397
  (0, 4725)	0.2529951875330347
  (0, 8021)	0.09053535321130156
  (0, 5822)	0.20917025071811787
  (0, 6733)	0.21714910913379717
  (0, 2686)	0.2762672188591472
  (0, 4070)	0.20106056057132318
  (0, 10935)	0.1473865119226825
  (0, 11950)	0.14366432547019645
  (0, 6393)	0.19415932690073123
  (0, 5466)	0.2292327981324229
  (0, 10614)	0.3125789825606384
  (0, 1207)	0.353251250137464
  (0, 4861)	0.09123170276992891
  (0, 9010)	0.17449986505465276
  (0, 6368)	0.10709591260647869
  (1, 3855)	0.38783131610689137
  (1, 10821)	0.23966931915720752
  (1, 2028)	0.15509118577211845
  (1, 1970)	0.30682212135714304
  (1, 10983)	0.08469176885500944
  (1, 8367)	0.22863518843116257
  :	:
  (10246, 5839)	0.18127344436436105
  (10246, 11456)	0.10169412220599004
  (10246, 2697)	0.1525696793172952
  (10246, 10983)	0.08052757743297828
  (10246, 12261)	0.10429572341895883
  (10247, 11213)

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
df=data
df.drop(columns=['tweet_id','airline_sentiment_gold','tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone'])
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['text'])
y = df['airline_sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv("classification_report.csv")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

    negative       0.76      0.97      0.86      2814
     neutral       0.69      0.39      0.50       884
    positive       0.87      0.40      0.55       694

    accuracy                           0.76      4392
   macro avg       0.78      0.59      0.63      4392
weighted avg       0.77      0.76      0.74      4392

